In [1]:
!pip install -U libsvm-official
!pip install numpy matplotlib pandas

  Preparing metadata (setup.py) ... done
  Created wheel for libsvm-official: filename=libsvm_official-3.32.0-cp310-cp310-linux_x86_64.whl size=123884 sha256=956bd52e414edf29696b8a06664c644e95b9466ba9e95fcdcdc366c93dfcc119
  Stored in directory: /root/.cache/pip/wheels/61/3b/1b/73bb4869517f96a26c82b47ccdb9ec48f12f4466de2371eff6
Successfully built libsvm-official


In [108]:
# p9

from libsvm.svmutil import *
import numpy as np


def transform_X(x):
  X = np.array([0,0,0,0,0,0,0,0])
  # X = np.empty(shape = (len(x), 8))
  for i in range(len(x)):
    dic = x[i]
    rows = [value for key,value in dic.items()]
    X = np.vstack([X, rows])

  X = np.delete(X,0,0)
  return X


def compute_squared_error(y_true):
    return np.mean((y_true - np.mean(y_true)) ** 2)


def find_best_split(data, target):
    # Ensure data is a 2D numpy array and target is a 1D numpy array
    if len(data.shape) != 2 or len(target.shape) != 1:
        raise ValueError("Data must be a 2D array and target must be a 1D array")

    n_features = data.shape[1]
    best_feature, best_theta, min_error = None, None, float('inf')

    for feature in range(n_features):
        sorted_values = np.unique(np.sort(data[:, feature]))
        thetas = (sorted_values[:-1] + sorted_values[1:]) / 2

        for theta in thetas:
            left_mask = data[:, feature] <= theta
            right_mask = ~left_mask

            left_target = target[left_mask]
            right_target = target[right_mask]

            left_error = compute_squared_error(left_target) if left_target.size > 0 else 0
            right_error = compute_squared_error(right_target) if right_target.size > 0 else 0

            total_error = left_error * left_mask.sum() + right_error * right_mask.sum()

            if total_error < min_error:
                best_feature, best_theta, min_error = feature, theta, total_error

    return best_feature, best_theta


class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeRegressor:
    def __init__(self, depth = None):
        self.depth = depth
        self.tree = {}

    def fit(self, data, target, depth=0):
      # self.root = self.fit(data, target)
        if depth == self.depth or data.shape[0] < 2:
            self.tree['value'] = target.mean()
            return

        best_feature, best_theta = find_best_split(data, target)
        if best_feature is None:
            self.tree['value'] = target.mean()
            return

        self.tree = {'feature': best_feature, 'theta': best_theta, 'left': {}, 'right': {}}

        left_mask = data[:, best_feature] <= best_theta
        right_mask = ~left_mask

        left_subtree = DecisionTreeRegressor(self.depth)
        left_subtree.fit(data[left_mask], target[left_mask], depth + 1)
        self.tree['left'] = left_subtree.tree

        right_subtree = DecisionTreeRegressor(self.depth)
        right_subtree.fit(data[right_mask], target[right_mask], depth + 1)
        self.tree['right'] = right_subtree.tree


    def predict(self, X):
        # Method to handle multiple instances (rows in 2D array)
        if len(X.shape) != 2:
            raise ValueError("Predict method expects a 2D array of instances")
        return np.array([self._predict_recursive(x, self.tree) for x in X])

    def _predict_recursive(self, x, tree):
        # Internal method for recursive prediction for a single instance
        if 'value' in tree:
            return tree['value']

        feature, theta = tree['feature'], tree['theta']
        if x[feature] <= theta:
            return self._predict_recursive(x, tree['left'])
        else:
            return self._predict_recursive(x, tree['right'])

    # def _predict_recursive(self, x, node):
    #     # Internal method for recursive prediction for a single instance
    #     if node.value is not None:
    #         return node.value

    #     if x[node.feature_index] <= node.threshold:
    #         return self._predict_recursive(x, node.left)
    #     else:
    #         return self._predict_recursive(x, node.right)



def main():
  train_y , train_x = svm_read_problem("train.dat")
  test_y, test_x = svm_read_problem("test.dat")
  train_x = transform_X(train_x)
  test_x = transform_X(test_x)

  train_x = np.array(train_x)
  train_y = np.array(train_y)
  test_x = np.array(test_x)
  test_y = np.array(test_y)

  # Train model
  tree = DecisionTreeRegressor()
  tree.fit(train_x, train_y)

  # Evaluate model
  predictions = tree.predict(test_x)
  squared_error = np.mean((y_test - predictions) ** 2)
  print("Eout: ", squared_error)
  # print(x)
  # print(y)


main()




Eout:  8.735926305015353


In [ ]:
# p10

from libsvm.svmutil import *
import numpy as np
import matplotlib.pyplot as plt


def transform_X(x):
  X = np.array([0,0,0,0,0,0,0,0])
  # X = np.empty(shape = (len(x), 8))
  for i in range(len(x)):
    dic = x[i]
    rows = [value for key,value in dic.items()]
    X = np.vstack([X, rows])

  X = np.delete(X,0,0)
  return X


def compute_squared_error(y_true):
    return np.mean((y_true - np.mean(y_true)) ** 2)


def find_best_split(data, target):
    # Ensure data is a 2D numpy array and target is a 1D numpy array
    if len(data.shape) != 2 or len(target.shape) != 1:
        raise ValueError("Data must be a 2D array and target must be a 1D array")

    n_features = data.shape[1]
    best_feature, best_theta, min_error = None, None, float('inf')

    for feature in range(n_features):
        sorted_values = np.unique(np.sort(data[:, feature]))
        thetas = (sorted_values[:-1] + sorted_values[1:]) / 2

        for theta in thetas:
            left_mask = data[:, feature] <= theta
            right_mask = ~left_mask

            left_target = target[left_mask]
            right_target = target[right_mask]

            left_error = compute_squared_error(left_target) if left_target.size > 0 else 0
            right_error = compute_squared_error(right_target) if right_target.size > 0 else 0

            total_error = left_error * left_mask.sum() + right_error * right_mask.sum()

            if total_error < min_error:
                best_feature, best_theta, min_error = feature, theta, total_error

    return best_feature, best_theta


class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeRegressor:
    def __init__(self, depth = None):
        self.depth = depth
        self.tree = {}

    def fit(self, data, target, depth=0):
      # self.root = self.fit(data, target)
        if depth == self.depth or data.shape[0] < 2:
            self.tree['value'] = target.mean()
            return

        best_feature, best_theta = find_best_split(data, target)
        if best_feature is None:
            self.tree['value'] = target.mean()
            return

        self.tree = {'feature': best_feature, 'theta': best_theta, 'left': {}, 'right': {}}

        left_mask = data[:, best_feature] <= best_theta
        right_mask = ~left_mask

        left_subtree = DecisionTreeRegressor(self.depth)
        left_subtree.fit(data[left_mask], target[left_mask], depth + 1)
        self.tree['left'] = left_subtree.tree

        right_subtree = DecisionTreeRegressor(self.depth)
        right_subtree.fit(data[right_mask], target[right_mask], depth + 1)
        self.tree['right'] = right_subtree.tree


    def predict(self, X):
        # Method to handle multiple instances (rows in 2D array)
        if len(X.shape) != 2:
            raise ValueError("Predict method expects a 2D array of instances")
        return np.array([self._predict_recursive(x, self.tree) for x in X])

    def _predict_recursive(self, x, tree):
        # Internal method for recursive prediction for a single instance
        if 'value' in tree:
            return tree['value']

        feature, theta = tree['feature'], tree['theta']
        if x[feature] <= theta:
            return self._predict_recursive(x, tree['left'])
        else:
            return self._predict_recursive(x, tree['right'])

    # def _predict_recursive(self, x, node):
    #     # Internal method for recursive prediction for a single instance
    #     if node.value is not None:
    #         return node.value

    #     if x[node.feature_index] <= node.threshold:
    #         return self._predict_recursive(x, node.left)
    #     else:
    #         return self._predict_recursive(x, node.right)

def main():
    train_y, train_x = svm_read_problem("train.dat")
    test_y, test_x = svm_read_problem("test.dat")
    train_x = transform_X(train_x)
    test_x = transform_X(test_x)

    train_x = np.array(train_x)
    train_y = np.array(train_y)
    test_x = np.array(test_x)
    test_y = np.array(test_y)

    num_trees = 2000
    e_outs = []

    for i in range(num_trees):
        # Bagging: Sample with replacement
        indices = np.random.choice(len(train_x), size=int(0.5 * len(train_x)), replace=True)
        bagged_x = train_x[indices]
        bagged_y = train_y[indices]

        # Train model
        tree = DecisionTreeRegressor()
        tree.fit(bagged_x, bagged_y)

        # Evaluate model
        predictions = tree.predict(test_x)
        squared_error = np.mean((test_y - predictions) ** 2)
        e_outs.append(squared_error)
        print("Random_Forest( {}/2000 )".format(i+1))

    # Plot histogram
    plt.hist(e_outs, bins=30, edgecolor='black')
    plt.xlabel('E_out')
    plt.ylabel('Frequency')
    plt.title('Histogram of E_out for 2000 Trees in Random Forest')
    plt.show()

main()

Random_Forest( 1/2000 )
Random_Forest( 2/2000 )
Random_Forest( 3/2000 )
Random_Forest( 4/2000 )
Random_Forest( 5/2000 )
Random_Forest( 6/2000 )
Random_Forest( 7/2000 )
Random_Forest( 8/2000 )
Random_Forest( 9/2000 )
Random_Forest( 10/2000 )
Random_Forest( 11/2000 )
Random_Forest( 12/2000 )
Random_Forest( 13/2000 )
Random_Forest( 14/2000 )
Random_Forest( 15/2000 )
Random_Forest( 16/2000 )
Random_Forest( 17/2000 )
Random_Forest( 18/2000 )
Random_Forest( 19/2000 )
Random_Forest( 20/2000 )
Random_Forest( 21/2000 )
Random_Forest( 22/2000 )
Random_Forest( 23/2000 )
Random_Forest( 24/2000 )
Random_Forest( 25/2000 )
Random_Forest( 26/2000 )
Random_Forest( 27/2000 )
Random_Forest( 28/2000 )
Random_Forest( 29/2000 )
Random_Forest( 30/2000 )
Random_Forest( 31/2000 )
Random_Forest( 32/2000 )
Random_Forest( 33/2000 )
Random_Forest( 34/2000 )
Random_Forest( 35/2000 )
Random_Forest( 36/2000 )
Random_Forest( 37/2000 )
Random_Forest( 38/2000 )
Random_Forest( 39/2000 )
Random_Forest( 40/2000 )
Random_Fo

In [ ]:
# p11

from libsvm.svmutil import *
import numpy as np
import matplotlib.pyplot as plt


def transform_X(x):
  X = np.array([0,0,0,0,0,0,0,0])
  # X = np.empty(shape = (len(x), 8))
  for i in range(len(x)):
    dic = x[i]
    rows = [value for key,value in dic.items()]
    X = np.vstack([X, rows])

  X = np.delete(X,0,0)
  return X


def compute_squared_error(y_true):
    return np.mean((y_true - np.mean(y_true)) ** 2)


def find_best_split(data, target):
    # Ensure data is a 2D numpy array and target is a 1D numpy array
    if len(data.shape) != 2 or len(target.shape) != 1:
        raise ValueError("Data must be a 2D array and target must be a 1D array")

    n_features = data.shape[1]
    best_feature, best_theta, min_error = None, None, float('inf')

    for feature in range(n_features):
        sorted_values = np.unique(np.sort(data[:, feature]))
        thetas = (sorted_values[:-1] + sorted_values[1:]) / 2

        for theta in thetas:
            left_mask = data[:, feature] <= theta
            right_mask = ~left_mask

            left_target = target[left_mask]
            right_target = target[right_mask]

            left_error = compute_squared_error(left_target) if left_target.size > 0 else 0
            right_error = compute_squared_error(right_target) if right_target.size > 0 else 0

            total_error = left_error * left_mask.sum() + right_error * right_mask.sum()

            if total_error < min_error:
                best_feature, best_theta, min_error = feature, theta, total_error

    return best_feature, best_theta


class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeRegressor:
    def __init__(self, depth = None):
        self.depth = depth
        self.tree = {}

    def fit(self, data, target, depth=0):
      # self.root = self.fit(data, target)
        if depth == self.depth or data.shape[0] < 2:
            self.tree['value'] = target.mean()
            return

        best_feature, best_theta = find_best_split(data, target)
        if best_feature is None:
            self.tree['value'] = target.mean()
            return

        self.tree = {'feature': best_feature, 'theta': best_theta, 'left': {}, 'right': {}}

        left_mask = data[:, best_feature] <= best_theta
        right_mask = ~left_mask

        left_subtree = DecisionTreeRegressor(self.depth)
        left_subtree.fit(data[left_mask], target[left_mask], depth + 1)
        self.tree['left'] = left_subtree.tree

        right_subtree = DecisionTreeRegressor(self.depth)
        right_subtree.fit(data[right_mask], target[right_mask], depth + 1)
        self.tree['right'] = right_subtree.tree


    def predict(self, X):
        # Method to handle multiple instances (rows in 2D array)
        if len(X.shape) != 2:
            raise ValueError("Predict method expects a 2D array of instances")
        return np.array([self._predict_recursive(x, self.tree) for x in X])

    def _predict_recursive(self, x, tree):
        # Internal method for recursive prediction for a single instance
        if 'value' in tree:
            return tree['value']

        feature, theta = tree['feature'], tree['theta']
        if x[feature] <= theta:
            return self._predict_recursive(x, tree['left'])
        else:
            return self._predict_recursive(x, tree['right'])

    # def _predict_recursive(self, x, node):
    #     # Internal method for recursive prediction for a single instance
    #     if node.value is not None:
    #         return node.value

    #     if x[node.feature_index] <= node.threshold:
    #         return self._predict_recursive(x, node.left)
    #     else:
    #         return self._predict_recursive(x, node.right)

def main():
    train_y, train_x = svm_read_problem("train.dat")
    test_y, test_x = svm_read_problem("test.dat")
    train_x = transform_X(train_x)
    test_x = transform_X(test_x)

    train_x = np.array(train_x)
    train_y = np.array(train_y)
    test_x = np.array(test_x)
    test_y = np.array(test_y)

    num_trees = 2000
    e_in_trees = []
    e_out_trees = []
    trees = []

    for i in range(num_trees):
        # Bagging: Sample with replacement
        indices = np.random.choice(len(train_x), size=int(0.5 * len(train_x)), replace=True)
        bagged_x = train_x[indices]
        bagged_y = train_y[indices]

        # Train model
        tree = DecisionTreeRegressor()
        tree.fit(bagged_x, bagged_y)
        trees.append(tree)

        # Evaluate model
        predictions_train = tree.predict(train_x)
        predictions_test = tree.predict(test_x)

        e_in = np.mean((train_y - predictions_train) ** 2)
        e_out = np.mean((test_y - predictions_test) ** 2)

        e_in_trees.append(e_in)
        e_out_trees.append(e_out)
        print("Random_Forest( {}/2000 )".format(i+1))


    # Calculate E_in(G) and E_out(G)
    predictions_train_ensemble = np.mean([tree.predict(train_x) for tree in trees], axis=0)
    predictions_test_ensemble = np.mean([tree.predict(test_x) for tree in trees], axis=0)

    e_in_ensemble = np.mean((train_y - predictions_train_ensemble) ** 2)
    e_out_ensemble = np.mean((test_y - predictions_test_ensemble) ** 2)

    # Scatter plot of (Ein(gt), Eout(gt))
    plt.scatter(e_in_trees, e_out_trees, alpha=0.6, label='Individual Trees')
    plt.scatter(e_in_ensemble, e_out_ensemble, color='red', label='Random Forest', zorder=5)
    plt.xlabel('Ein(gt)')
    plt.ylabel('Eout(gt)')
    plt.title('Scatter Plot of Ein(gt) vs Eout(gt) for Random Forest')
    plt.legend()
    plt.show()

main()


In [ ]:
# p12

from libsvm.svmutil import *
import numpy as np
import matplotlib.pyplot as plt


def transform_X(x):
  X = np.array([0,0,0,0,0,0,0,0])
  # X = np.empty(shape = (len(x), 8))
  for i in range(len(x)):
    dic = x[i]
    rows = [value for key,value in dic.items()]
    X = np.vstack([X, rows])

  X = np.delete(X,0,0)
  return X


def compute_squared_error(y_true):
    return np.mean((y_true - np.mean(y_true)) ** 2)


def find_best_split(data, target):
    # Ensure data is a 2D numpy array and target is a 1D numpy array
    if len(data.shape) != 2 or len(target.shape) != 1:
        raise ValueError("Data must be a 2D array and target must be a 1D array")

    n_features = data.shape[1]
    best_feature, best_theta, min_error = None, None, float('inf')

    for feature in range(n_features):
        sorted_values = np.unique(np.sort(data[:, feature]))
        thetas = (sorted_values[:-1] + sorted_values[1:]) / 2

        for theta in thetas:
            left_mask = data[:, feature] <= theta
            right_mask = ~left_mask

            left_target = target[left_mask]
            right_target = target[right_mask]

            left_error = compute_squared_error(left_target) if left_target.size > 0 else 0
            right_error = compute_squared_error(right_target) if right_target.size > 0 else 0

            total_error = left_error * left_mask.sum() + right_error * right_mask.sum()

            if total_error < min_error:
                best_feature, best_theta, min_error = feature, theta, total_error

    return best_feature, best_theta


class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeRegressor:
    def __init__(self, depth = None):
        self.depth = depth
        self.tree = {}

    def fit(self, data, target, depth=0):
      # self.root = self.fit(data, target)
        if depth == self.depth or data.shape[0] < 2:
            self.tree['value'] = target.mean()
            return

        best_feature, best_theta = find_best_split(data, target)
        if best_feature is None:
            self.tree['value'] = target.mean()
            return

        self.tree = {'feature': best_feature, 'theta': best_theta, 'left': {}, 'right': {}}

        left_mask = data[:, best_feature] <= best_theta
        right_mask = ~left_mask

        left_subtree = DecisionTreeRegressor(self.depth)
        left_subtree.fit(data[left_mask], target[left_mask], depth + 1)
        self.tree['left'] = left_subtree.tree

        right_subtree = DecisionTreeRegressor(self.depth)
        right_subtree.fit(data[right_mask], target[right_mask], depth + 1)
        self.tree['right'] = right_subtree.tree


    def predict(self, X):
        # Method to handle multiple instances (rows in 2D array)
        if len(X.shape) != 2:
            raise ValueError("Predict method expects a 2D array of instances")
        return np.array([self._predict_recursive(x, self.tree) for x in X])

    def _predict_recursive(self, x, tree):
        # Internal method for recursive prediction for a single instance
        if 'value' in tree:
            return tree['value']

        feature, theta = tree['feature'], tree['theta']
        if x[feature] <= theta:
            return self._predict_recursive(x, tree['left'])
        else:
            return self._predict_recursive(x, tree['right'])

    # def _predict_recursive(self, x, node):
    #     # Internal method for recursive prediction for a single instance
    #     if node.value is not None:
    #         return node.value

    #     if x[node.feature_index] <= node.threshold:
    #         return self._predict_recursive(x, node.left)
    #     else:
    #         return self._predict_recursive(x, node.right)

def main():
    train_y, train_x = svm_read_problem("train.dat")
    test_y, test_x = svm_read_problem("test.dat")
    train_x = transform_X(train_x)
    test_x = transform_X(test_x)

    train_x = np.array(train_x)
    train_y = np.array(train_y)
    test_x = np.array(test_x)
    test_y = np.array(test_y)

    num_trees = 2000
    e_out_individual_trees = []
    e_out_cumulative_forest = []
    cumulative_predictions = np.zeros_like(test_y, dtype=float)

    for i in range(num_trees):
        # Bagging: Sample with replacement
        indices = np.random.choice(len(train_x), size=int(0.5 * len(train_x)), replace=True)
        bagged_x = train_x[indices]
        bagged_y = train_y[indices]

        # Train model
        tree = DecisionTreeRegressor()
        tree.fit(bagged_x, bagged_y)

        # Evaluate individual tree model
        predictions = tree.predict(test_x)
        squared_error = np.mean((test_y - predictions) ** 2)
        e_out_individual_trees.append(squared_error)

        # Update cumulative predictions and calculate error for cumulative forest
        cumulative_predictions += predictions
        average_cumulative_predictions = cumulative_predictions / (i + 1)
        cumulative_forest_error = np.mean((test_y - average_cumulative_predictions) ** 2)
        e_out_cumulative_forest.append(cumulative_forest_error)

        print("Random Forest Progress: {}/2000 Trees".format(i + 1))

    # Plotting
    plt.plot(range(1, num_trees + 1), e_out_individual_trees, label='E_out(gt)')
    plt.plot(range(1, num_trees + 1), e_out_cumulative_forest, label='E_out(Gt)')
    plt.xlabel('Number of Trees (t)')
    plt.ylabel('E_out')
    plt.title('E_out(gt) and E_out(Gt) as a Function of t')
    plt.legend()
    plt.show()

main()
